<a href="https://colab.research.google.com/github/princedl/collab-java/blob/master/Chroma_Studio_1_EngineIV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LOCAL
---
```
jupyter-notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

mp3 Custom Tags
https://stackoverflow.com/questions/45201589/custom-key-in-mp3-file-metadata
```

In [ ]:
#This is new and newer

In [ ]:
#sudo apt-get install rubberband-cli

In [ ]:
#!pip3 install --upgrade ipython

In [ ]:
!pip3 install librosa --upgrade
!pip3 install matplotlib
!pip3 install tqdm
!pip3 install Cython
!pip3 install madmom
#PyDub https://medium.com/better-programming/simple-audio-processing-in-python-with-pydub-c3a217dabf11
!pip3 install pydub
#!pip3 install -U ipython-namespaces
#%load_ext ipython_namespaces

In [ ]:
import librosa
import librosa.display
import numpy as np
from pathlib import Path
from urllib.parse import unquote
import os.path
import IPython
from IPython.display import Audio
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import math
from pydub import AudioSegment
import madmom
import soundfile as sf
import os
import xml.etree.ElementTree as ET
import mutagen


In [ ]:
librosa.__version__

#TRACK

In [ ]:
def computeBPM_s(start, end, beats=32):
  return 60/((end-start)/beats)

In [ ]:
def exportCueMeasures(t, measures):
  print(t)
  ebg = BeatGrid.fromTrack(t)
  fb = ebg.get111ind()
  for i, m in enumerate(measures):
    cue = f'<Poi Name="{m}" Pos="{ebg.getBeat_p(fb+4*m):.6f}" Num="{i+1}" Type="cue" />'
    print(cue)

In [ ]:
def exportCueBeats(t, beats):
  print(t)
  ebg = BeatGrid.fromTrack(t)
  for i, b in enumerate(beats):
    cue = f'<Poi Name="{b}" Pos="{ebg.getBeat_p(b):.6f}" Num="{i+1}" Type="cue" />'
    print(cue)

In [ ]:
def getDownbeats(res, dbi=1):
  dbs = [t for t, b in res if int(b)==dbi]
  return dbs

In [ ]:
def exportBeatgridVDJ(t, delta=0.0, dbi=1):
  ebg = BeatGrid.fromTrack(t)
  dbs = ebg.getDownBeatTimes(dbi)
  for i, db in enumerate(dbs[:-1]):
    st = db+delta
    et = dbs[i+1]+delta

    bpm = computeBPM_s(st, et, beats=4)
    bpmr = round(bpm,3)-.005
    etprime = st+60/bpmr*4
    if etprime < et:
      print('et=', et, '; etp=', etprime)

    poi = f'<Poi Pos="{st:.6f}" Type="beatgrid" Bpm="{bpmr:.3f}000"/>'
    print(poi)

In [ ]:
def getComment(track):
  res = mutagen.File(track)
  if 'COMM::eng' in res:
      return res['COMM::eng'].text[0]
  else:
    return None

In [ ]:
def primarykey(track):
  return str(track)

In [ ]:
def getMixedInKeyKey(track):
  comment = getComment(track)
  if comment is not None:
    mik = comment.split(';')[0]
    key = mik.split('-')[0]
    return key
  else:
    return None

In [ ]:
def lookup(pk):
  for t in tracks:
    if primarykey(t)==pk:
      return t

#GENERIC

In [ ]:
def loadMadmomBeats(fn):
  beats = list()

  try:
    f = open(fn)
  except IOError:
    print('error')
    return beats
  else:
    with f:
      for line in f:
        parts = line.split('\t')
        t = float(parts[0])
        beat = int(parts[1].strip())
        #print(t, beat)
        beats.append((t, beat))
      return beats

In [ ]:
#FAKE!!!!!!!!!!!!!!!!
#class MadmomBeatGrid():
class BeatGrid():

  @classmethod
  def fromTrack(cls, track):
    return cls(track)


  

  def __init__(self, track):
    self.track = track

    #fn = getWSLFilename(track)
    fn = str(track)
    fn = fn.replace('mp3', 'beats')
    print(fn)
    self.beats = loadMadmomBeats(fn)
    if(len(self.beats)==0):
      fn = str(track)
      fn = fn+'.beats'
      print(fn)
      self.beats = loadMadmomBeats(fn)



    #Store cue/and loops - code is ugly
    self.loops = dict()
    self.cues = dict()

    # for pm in track.findall('POSITION_MARK'):
    #   if pm.attrib['Type'] == '4':
    #     self.loops[pm.attrib['Name']]=pm.attrib
    #   else: 
    #     self.cues[pm.attrib['Name']]=pm.attrib
  

  def getNearestBeat(self, position):
    mind = 99999.
    mini = -1
    for i, b in enumerate(self.beats):
      pos, _= b
      d = abs(pos - position)
      if(d < mind):
        mind = d
        mini = i
    return mini, mind


  def get111ind(self):
    for i in range(4):
      _, b = self.beats[i]
      if b == 1:
        return i
    return -1


  def getLastBeatInd(self):
    fb = self.get111ind()
    _, b  = self.beats[fb]
    #Get the last one
    last = -1
    for i in range(fb, len(self.beats)):
      if self.beats[i][1]==b:
        last = i
    return last



  def getBeat_p(self, i):
    p, _ = self.beats[i]
    return p



  def getBeatTimes(self):
    beattimes = list()
    for b in self.beats:
      t, _ = b
      beattimes.append(t)
    return beattimes

  def getDownBeatTimes(self, db_ind=1):
    beattimes = list()
    for b in self.beats:
      t, beat = b
      #print(t, beat)
      if beat==db_ind:
        beattimes.append(t)
    return beattimes


  #convenience :)
  def loop_s(self, name):
    return float(self.loops[name]['Start'])
  
  def loop_e(self, name):
    return float(self.loops[name]['End'])

  def cue_p(self, name):
    return float(self.cues[name]['Start'])


  def cueBind(self):
    if 'b' in self.cues:
      t = self.cue_p('b')
      i, _ = self.getNearestBeat(t)
      return i
    else:
      return None

  
  def cueQind(self, q):
    for pm in self.track.findall('POSITION_MARK'):
      if pm.attrib['Name'].startswith(q):
        t = float(pm.attrib['Start'])
        i, _ = self.getNearestBeat(t)
        return i
        
    return None

# Process Tracks

In [ ]:
def isDebug(track):
  # #print(track.attrib['TrackID'])
  # if track.attrib['TrackID']=='121212126':
  #   return True
  return False

Taken from https://stackoverflow.com/questions/57082826/how-can-a-chromagram-file-produced-by-librosa-be-interpreted-as-a-set-of-musical

In [ ]:
def plotChromas(chromas):
  fig, ax = plt.subplots(figsize=(30, 10))
  img = librosa.display.specshow(chromas, y_axis='chroma', x_axis='time', ax=ax)
  fig.colorbar(img, ax=ax)
  ax.set(title='Chromagram')

In [ ]:
def processTrack(track):
  
  filename = str(track)
  if(os.path.isfile(filename)):
    print(filename + " exists, will be processed.")
  


    bg = BeatGrid(track)

    firstBeat = bg.get111ind()
    lastBeat = bg.getLastBeatInd()
    if bg.cueBind() is not None:
      firstBeat = bg.cueBind()

    y, sr = librosa.load(str(filename))
    #y, sr = librosa.load(MIXX / filename)
    
    chromas = librosa.feature.chroma_stft(y=y, sr=sr)


    beats = bg.getBeatTimes()

    medians = list()
    for i in range(lastBeat-firstBeat):
      st = beats[firstBeat + i]
      et = beats[firstBeat + i + 1]
      median = np.median(chromas[:,librosa.time_to_frames(st):librosa.time_to_frames(et)], axis = 1)
      median = median / np.linalg.norm(median)
      medians.append(median)

    medians16 = list()
    for i in range(lastBeat-firstBeat):
      st = beats[firstBeat + i]
      et = beats[firstBeat + i + 1]
      delta = et - st
      for j in range(4):
        st16 = st+j*delta/4
        et16 = st16+delta/4
        #print(st, et ,st16, et16)
        median16 = np.median(chromas[:,librosa.time_to_frames(st16):librosa.time_to_frames(et16)], axis = 1)
        median16 = median16 / np.linalg.norm(median16)
        medians16.append(median16)


    downbeats = bg.getDownBeatTimes()
    medians1 = list()
    for i in range(len(downbeats)-1):
      st1 = downbeats[i]
      et1 = downbeats[i + 1]
      median1 = np.median(chromas[:,librosa.time_to_frames(st1):librosa.time_to_frames(et1)], axis = 1)
      median1 = median1 / np.linalg.norm(median1)
      medians1.append(median1)

    print(4)


    if isDebug(track):
      for i in range(16):
        plotChromas(chromas[:,_65beats[i]:_65beats[i+1]])

    print(filename + " DONE.")
    return medians, medians1, medians16



In [ ]:
!ls /mnt/c//Users/andre/OneDrive/Music/Library/5fevftlD8al7MLhmBE9bQfX/'Rick Astley - Never Gonna Give You Up.mp3'

MAIN LOOP


---



In [ ]:
def dump(fn, l):
  with open(fn, 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(l, filehandle)

In [ ]:
import warnings

#ignore by message
#warnings.filterwarnings("ignore", message="PySoundFile failed.")

#part of the message is also okay
#warnings.filterwarnings("ignore", message="divide by zero encountered") 
#warnings.filterwarnings("ignore", message="invalid value encountered")

In [ ]:
for path in tqdm(list(Path('/mnt/c/Users/andre/OneDrive/Music/Library/').rglob('*.mp3'))):
  dumpfile = Path(f'{str(path)}-pair.m4116Ldata')
  if not dumpfile.exists():

    m4, m1, m16 = None, None, None
    #try:
    m4, m1, m16 = processTrack(path)
    # except Exception as e:
    #   print(repr(e))
    #   print(f'ERROR PROCESSING {dumpfile} , skipping.......')
    #   continue

    
    #Skip if no mp3 is found
    if m4 is None:
      print('ALARM, m4 is None')
      continue

    triplet = (m4, m1, m16)
    dump(dumpfile, triplet)

    #trackMedians.extend(m)
    #trackNames.extend(names)




In [ ]:
BREAK()
for track in tqdm(tracks):
  filename = getWSLFilename(track)
  dumpfilename = Path(f'{filename}-pair.m4116data')
  #Skip if already processed
  # if dumpfilename.exists():
  #   print(f'{dumpfilename} already exists, skipping.......')
  #   continue

  if not isYellow(track):
    continue

  m4, m1, m16 = None, None, None
  try:
    m4, m1, m16 = processTrack(track)
  except Exception as e:
    print(repr(e))
    print(f'ERROR PROCESSING {dumpfilename} , skipping.......')
    continue

  
  #Skip if no mp3 is found
  if m4 is None:
    continue

  triplet = (m4, m1, m16)
  dump(dumpfilename, triplet)

  #trackMedians.extend(m)
  #trackNames.extend(names)

#Use Chromas
Reload data and query it!



# Neuer Abschnitt

In [ ]:
def createPitchedAudio(track, duration, which):

  pitches = [0, -1, -2, -3, 1, 2, 3]
  witches = [0, 1, 2, 3, 4, 5, 6]
  pa = list()


  audio = AudioSegment.from_file(track, format="mp3")
  audio[0:duration*1000].export(f'audio-00.wav', format='wav')

  for i, p in enumerate(pitches):
    if i in which:
      rubbertext = f'rubberband --tempo 1. --pitch {p}  audio-00.wav audio-00-{p}.wav'
      print(rubbertext)
      os.system(rubbertext)
      paudio = AudioSegment.from_file(f'audio-00-{p}.wav', format="wav")
      pa.append(paudio)
    else:
      pa.append(None)

  return pa


In [ ]:
%%time
TBP = createPitchedAudio(tb)

In [ ]:
def buildOverlayPitch(track, pattern, targetBPM, pitches, firstBeat=None):
  id = primarykey(track)

  bg = BeatGrid(track)
  if firstBeat is None:
    print("Firstbeat is NONE")
    firstBeat = bg.get111ind()

  firstbeat = firstBeat


  maxm = max(pattern)
  duration = bg.getBeat_p(firstbeat+maxm*4+8)
  pa = createPitchedAudio(track, duration, pitches)

  overlay = AudioSegment.empty()
  for i, b in enumerate(pattern):
    start = bg.getBeat_p(firstbeat+b*4)
    end   = bg.getBeat_p(firstbeat+b*4+4)

    start_ms = int(start*1000)
    end_ms   = int(end*1000)

    pitch = pitches[i]
    chop = pa[pitch][start_ms:end_ms]
    #IPython.display.display(chop)

    overlay += chop
  

  id = 'GENERIC'
  #IPython.display.display(overlay)
  overlay.export(f'{id}-overlay.wav', format='wav')

  bpm = computeBPM_s(0, len(overlay)/1000, beats=len(pattern)*4)
  print(f'Overlay BPM{bpm}')
  ratio = targetBPM/bpm

  # if pitchShift==5 or pitchShift==7:
  #   pitchShift=0
  # if pitchShift==8 or pitchShift==6:
  #   pitchShift=1
  # if pitchShift==4:
  #   pitchShift=-1

 

  rubbertext = f'rubberband --tempo {targetBPM/bpm} --pitch 0  {id}-overlay.wav {id}-overlay-rubber.wav'
  print(rubbertext)
  os.system(rubbertext)

  #TODO - Overlay Filename?
  overlay = AudioSegment.from_file(f'{id}-overlay-rubber.wav', format="wav")
  return overlay

In [ ]:
def buildOverlay(track, pattern, targetBPM, pitchShift=0, firstBeat=None):
  id = primarykey(track)

  bg = BeatGrid(track)
  if firstBeat is None:
    print("Firstbeat is NONE")
    firstBeat = bg.get111ind()

  firstbeat = firstBeat

  audio = AudioSegment.from_file(track, format="mp3")

  overlay = AudioSegment.empty()
  for b in pattern:
    start = bg.getBeat_p(firstbeat+b*4)
    end   = bg.getBeat_p(firstbeat+b*4+4)

    start_ms = int(start*1000)
    end_ms   = int(end*1000)
    chop = audio[start_ms:end_ms]
    #IPython.display.display(chop)

    overlay += chop
  

  id = 'GENERIC'
  #IPython.display.display(overlay)
  overlay.export(f'{id}-overlay.wav', format='wav')

  bpm = computeBPM_s(0, len(overlay)/1000, beats=len(pattern)*4)
  print(f'Overlay BPM{bpm}')
  ratio = targetBPM/bpm

  # if pitchShift==5 or pitchShift==7:
  #   pitchShift=0
  # if pitchShift==8 or pitchShift==6:
  #   pitchShift=1
  # if pitchShift==4:
  #   pitchShift=-1

  realPitchShift = [0, -1, -2, -3, 1, 0, -1, 0, -1, 3, 2, 1][pitchShift]
  print(f'Pitchshift: in:{pitchShift}, res:{realPitchShift}')

  rubbertext = f'rubberband --tempo {targetBPM/bpm} --pitch {realPitchShift}  {id}-overlay.wav {id}-overlay-rubber.wav'
  print(rubbertext)
  os.system(rubbertext)

  #TODO - Overlay Filename?
  overlay = AudioSegment.from_file(f'{id}-overlay-rubber.wav', format="wav")
  return overlay

In [ ]:
def outputMix(loop, score, tag, loopdb=-15, ovldb=-12, filename=None, hps=None, _pattternind=4, showmix = True):
  s_ = score
  t = lookup(s_[1])
  print('-', tag , '-')
  pattern = s_[_pattternind]
  overlay = buildOverlay(t, pattern, loop[1], s_[2])
  if hps is not None:
    overlay = overlay.high_pass_filter(hps)
  mix = (loop[0] + loopdb).overlay(overlay+ovldb)
  if showmix:
    IPython.display.display(mix)
  if filename is not None:
    mix.export(f'{filename}.wav', format='wav')
    print(f'Exported {filename}.')

  return mix


In [ ]:
def outputMixPitch(loop, score, tag, loopdb=-15, ovldb=-12, filename=None, hps=None, _pattternind=4, showmix = True):
  s_ = score
  t = lookup(s_[1])
  print('-', tag , '-')
  pattern = s_[_pattternind]
  overlay = buildOverlayPitch(t, pattern, loop[1], s_[4])
  if hps is not None:
    overlay = overlay.high_pass_filter(hps)
  mix = (loop[0] + loopdb).overlay(overlay+ovldb)
  if showmix:
    IPython.display.display(mix)
  if filename is not None:
    mix.export(f'{filename}.wav', format='wav')
    print(f'Exported {filename}.')

  return mix


In [ ]:
def loadPair(fn):
  with open(fn, 'rb') as filehandle:
    # read the data as binary data stream
    triplet = pickle.load(filehandle)
    return triplet

In [ ]:
BEATS=96+96

trackm4 = dict()   #---------------- Step 1
trackm1 = dict()
trackm16 = dict()
remove = list()
tracks = list()

for path in tqdm(list(Path('/mnt/c/Users/andre/OneDrive/Music/Library/').rglob('*.mp3'))):

  dumpfile = Path(f'{str(path)}-pair.m4116Ldata')
  if dumpfile.exists():
    id = str(path)
    #print(id)
    # if id=='6443':
    #   print('>'+id+'<')
    #print(f'Loading {dumpfilename}')
    m4, m1, m16 = loadPair(dumpfile)
    if len(m4)<BEATS:
      remove.append(path)
      continue
    tracks.append(path)
    trackm4[id]=np.stack(m4[0:BEATS])
    trackm1[id]=np.stack(m1[0:int(BEATS/4)])
    trackm16[id]=np.stack(m16[0:BEATS*16])
  else:
    remove.append(path)
    if fn.find('Library') > 0:
      print('Not found', dumpfile)
  
  


In [ ]:
len(remove)

In [ ]:
dur  = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
moll = ['cm','c#m','dm','d#m','em','fm','f#m','gm','g#m','am','a#m','bm']

In [ ]:
def normalize(x):
  return x/np.linalg.norm(x)

In [ ]:
C3 = normalize(np.array([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]))
cm3 = normalize(np.array([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]))

In [ ]:
def createPCP(chord):
  if chord in dur:
    ind = dur.index(chord)
    print('dur: ', ind)
    return np.roll(C3, ind)
  elif chord.lower() in moll:
    ind = moll.index(chord.lower())
    print('moll: ', ind)
    return np.roll(cm3, ind)
  else:
    print(chord, " not found")

In [ ]:
createPCP('G')
createPCP('K')
createPCP('g#m')

In [ ]:
C_Dur = 'C,dm,em,F,G,am,bm'.split(',')
c_Moll = 'cm,D,E,fm,gm,A,B'.split(',')

In [ ]:
pcpsC = np.stack([createPCP(c) for c in C_Dur])
pcpsCm = np.stack([createPCP(c) for c in c_Moll])

In [ ]:
def createPCPs(key):
  #Töne in den Tonleitern ==> Grundtöne der Akkorde
  C = np.array([1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1])==1
  cm = np.array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0])==1
  pitches = np.array(['C','C#','D','D#','E','F','F#','G','G#','A','A#','B'])
  if key in dur:
    ind = dur.index(key)
    print('dur tonleiter: ', ind)
    xx = np.roll(pitches, -ind)[C]
    xx[1]=xx[1]+'m'
    xx[2]=xx[2]+'m'
    xx[5]=xx[5]+'m'
    xx[6]=xx[6]+'0'
    return (np.roll(pcpsC, ind, axis=1), xx)
  elif key.lower() in moll:
    ind = moll.index(key.lower())
    print('moll tonleiter: ', ind)
    return (np.roll(pcpsCm,ind,axis=1),  np.roll(pitches, -ind)[cm])
  else:
    print(key, " not found")
    return (None, None)

In [ ]:
def createFreePCPS():
  allpcps =      [createPCP(c) for c in dur]
  allpcps.extend([createPCP(c) for c in moll])
  return np.stack(allpcps)

##trackm

In [ ]:
trackm=trackm4

In [ ]:
def matchPCPS(chromas, pcps):
  match = np.matmul(chromas, pcps.T)
  ind = np.argmax(match, axis=1)

  return np.stack(pcps[ind])

In [ ]:
freepcps = createFreePCPS()
trackp = dict()
trackf = dict()
for t in tqdm(tracks):
  key = getMixedInKeyKey(t)
  if key is None:
    continue
  key = key.split('/')[0] #C#/D#
  key = key.strip()
  #print(t, '>'+key+'<')
  pcps, x = createPCPs(key)
  if pcps is None:
    continue
  #print(x)
  id = str(t)
  chromas = trackm[id]
  trackp[id]=matchPCPS(chromas,pcps)
  trackf[id]=matchPCPS(chromas,freepcps)

LOAD THE QUERY SONG [32 BEATS]


---



In [ ]:
def processQueryLoop(x, _33beats):
  
  chromas = librosa.feature.chroma_stft(y=x, sr=sr)

  medians4 = list()
  for i in range(len(_33beats)-1):
    st = _33beats[i]
    et = _33beats[i + 1]
    median4 = np.median(chromas[:,librosa.time_to_frames(st):librosa.time_to_frames(et)], axis = 1)
    median4 = median4 / np.linalg.norm(median4)
    medians4.append(median4)

  medians16 = list()
  for i in range(len(_33beats)-1):
    st = _33beats[i]
    et = _33beats[i + 1]
    delta = et - st
    for j in range(4):
      st16 = st+j*delta/4
      et16 = st16+delta/4
      median16 = np.median(chromas[:,librosa.time_to_frames(st16):librosa.time_to_frames(et16)], axis = 1)
      median16 = median16 / np.linalg.norm(median16)
      medians16.append(median16)

  # downbeats = list()
  # for i in [0, 4, 8, 12, 16, 20, 24, 28, 32]:
  #   downbeats.append(_33beats[i])

  medians1 = list()
  for i in range(0, len(_33beats)-1, 4):
    #print('m1:',i)
    st1 = _33beats[i]
    et1 = _33beats[i + 4]
    median1 = np.median(chromas[:,librosa.time_to_frames(st1):librosa.time_to_frames(et1)], axis = 1)
    median1 = median1 / np.linalg.norm(median1)
    medians1.append(median1)






  return medians4, medians1, medians16


In [ ]:
#list(range(0, 32-1, 4))

##Q


In [ ]:
def findTrack(name):
  found = list()
  for track in tracks:
    if str(track).lower().find(name.lower()) > 0:
      found.append(track)
      print(track)
  return found

In [ ]:
found = findTrack("magdalena"); found

In [ ]:
qt = found[1]

In [ ]:
mbg = BeatGrid.fromTrack(qt)

In [ ]:
mbg.get111ind()

In [ ]:
mbg.getLastBeatInd()

In [ ]:
audio = AudioSegment.from_file(qt, format="mp3")

In [ ]:
x, sr = librosa.load(str(qt)) 

In [ ]:
bt1     = mbg.getDownBeatTimes(1)
bt2     = mbg.getDownBeatTimes(2)
bt3     = mbg.getDownBeatTimes(3)
bt4     = mbg.getDownBeatTimes(4)


#print(dbs_ind)
b1clicks = librosa.clicks(bt1, sr=sr, click_freq=1000, length=len(x))
b2clicks = librosa.clicks(bt2, sr=sr, click_freq=1500, length=len(x))
b3clicks = librosa.clicks(bt3, sr=sr, click_freq=2000, length=len(x))
b4clicks = librosa.clicks(bt4, sr=sr, click_freq=2500, length=len(x))

clicks = b1clicks + b2clicks + b3clicks + b4clicks
#clicks = b1clicks



#dbclicks = librosa.clicks(mbg.getDownBeatTimes(3), sr=sr, click_freq=2000, length=len(x))

sf.write('xplusclicks.wav', x+clicks, sr, subtype='PCM_24')

clickseg = AudioSegment.from_wav('xplusclicks.wav')
clickseg

In [ ]:
#28 // 65

In [ ]:
bt = 65
for bp, b in enumerate(mbg.beats):
  if abs(b[0]-bt) < 3:
    print(bp, b)

In [ ]:
qs_ = list()
qstart =  48
qend   = 112
qs_.append((qstart, qend))
qlen = (qend-qstart)//16
print(qlen)

In [ ]:
112-48

In [ ]:
def plotTransition(audio):
  plt.figure(figsize=(20,10))  #plt.subplots(len(channels), sharex=True, figsize=(50,100))
  #fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')


  channel_sounds = audio.split_to_mono()
  samples = [s.get_array_of_samples() for s in channel_sounds]

  fp_arr = np.array(samples).T.astype(np.float32)
  fp_arr /= np.iinfo(samples[0].typecode).max

  print(fp_arr.shape)
  plt.plot(fp_arr)
  plt.axvline(x=fp_arr.shape[0]/2)
  plt.show()

In [ ]:
def chopLoops(audio, bg, qs):
  loops   = list()
  loops16 = list()
  for q in qs:
    start_ms = int(bg.getBeat_p(q[0])*1000)
    end_ms   = int(bg.getBeat_p(q[1])*1000)

    loop = audio[start_ms:end_ms]
    bpm = computeBPM_s(start_ms/1000, end_ms/1000, beats=q[1]-q[0])

    loops.append((loop, bpm))

    for i, b in enumerate(range(q[0], q[1]-15, 16)):
      print('==>',i+1, b,b+16)
      start_ms = int(bg.getBeat_p(b)*1000) #+correct[i]
      end_ms   = int(bg.getBeat_p(b+16)*1000) #+correct[i+1]
      loop = audio[start_ms:end_ms]

      plotTransition(audio[end_ms-50:end_ms+50])
      bpm = computeBPM_s(start_ms/1000, end_ms/1000, beats=16)

      loops16.append((loop, bpm))
  
  return loops, loops16


In [ ]:
qs_

In [ ]:
loops, loops16 = chopLoops(audio, mbg, qs_)

In [ ]:
loops[0][0]

In [ ]:
qt

In [ ]:
qkey = getMixedInKeyKey(qt)
qkey = qkey.split('/')[0] #C#/D#
qkey = qkey.strip()
print(qt, '>'+qkey+'<')

In [ ]:
qpcps, qx = createPCPs(qkey)
qx

###Q change 4, 16 (beats)

In [ ]:
qchromas = list()
for q in qs_:
  _beat0ind = q[0]
  _33beats = mbg.getBeatTimes()[q[0]:q[1]+1]
  print(len(_33beats))

  qmedians4, qmedians1, qmedians16 = processQueryLoop(x, _33beats)

  qchroma = np.stack(qmedians4)
  qfree   = matchPCPS(qchroma,freepcps)
  qkey    = matchPCPS(qchroma,qpcps)
  qchromas.append((qchroma, qfree, qkey))

In [ ]:
qs_

In [ ]:
qmine = trackm[str(qt)][34:162]

In [ ]:
len(qmedians1)

###trackse

In [ ]:
trackse = trackf.keys()
valuese = [trackf[k] for k in trackse]
trackme = np.concatenate(valuese)

In [ ]:
qchroma_.shape

##ULTRAWIDE Q, with 2 Measures match
scores = list()
for pitch in tqdm([-3, -2, -1, 0, 1, 2, 3]):
  qr = np.roll(qchroma_, pitch, axis=1)
  s = np.matmul(trackme, qr.T)
  dims = qr.shape[0]
  beats = int(dims/16)
  tstart = 0
  for track in trackse:
    score = 0
    pattern = list()
    for b in range(0,beats, 2):
      max     = -1.0
      maxbeat = -1
      for m_, measure in enumerate(range(tstart, tstart+384, 32)):
        #print(measure)
        x = np.trace(s[measure:measure+32, b*16:(b+2)*16]) #Was 0:16
        if x > max:
          max = x
          maxbeat = m_*2
      #print(maxbeat, x)
      score += max
      pattern.append(maxbeat)
      pattern.append(maxbeat+1)
    if not np.isnan(score):
      scores.append((score, track, -pitch, pattern))
    else:
      print('isnan')
    tstart+=384    

###ultraWideQ8->2MeasuresMatch

In [ ]:
BEATS//4-8+1

In [ ]:
#measureToIndex
def m2i(m, divisionsPerBeat):
  return int(m*4*divisionsPerBeat)

In [ ]:
#EX ULTRAWIDE Q, with 2 Measures match
def NOSIR-ultraWideQ2MeasuresMatch(trackse, trackme, qchroma_, divisionsPerBeat=1):
  dpb = divisionsPerBeat
  scores = list()
  qbeats = int(qchroma_.shape[0]//divisionsPerBeat)
  qmeasures = qbeats // 4

  print('qbeats:=', qbeats, '; qmeasures:=', qmeasures)

  NUMTRACKS=len(trackse)

  for w in tqdm(range(0,NUMTRACKS,500)):  
    for pitch in tqdm([0]):
      wend = min(w+500, NUMTRACKS)
      print(w,wend)
      qr = np.roll(qchroma_, pitch, axis=1)
      s = np.matmul(trackme[w*int(BEATS*divisionsPerBeat):wend*int(BEATS*divisionsPerBeat),:], qr.T)
      #print(trackme.shape, qr.shape, s.shape)
      tstart = 0
      for ti, track in enumerate(list(trackse)[w:wend]):
        #print('********************* New Track ************')
        gscore = 0
        patterns = list()
        pattern8s = list()
        for length in [8, 4]:
          score = 0
          pattern = list()
          pattern8 = list()
          #print('qm', 0,qmeasures, length)
          for qm in range(0,qmeasures, length):
            max         = -1.0
            maxmeasure  = -1
            max8        = -1.0
            maxmeasure8 = -1
            for tm in range(0,BEATS//4-length+1,4): #Welche TMs werden in Betracht gezogen?
              #print(tstart, tm, m2i(tm, dpb))
              x = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+length, dpb), m2i(qm, dpb):m2i(qm+length, dpb)])
              #print(ti, length, 'qm=', qm, 'tm=', tm, x)
              # if tm==0:
              #   print(ti, track, pitch, 'qm',qm, 'tm',tm, 'tstart', tstart, x)
              #print(tm, qm, x, m2i(tm, dpb), m2i(qm, dpb))
              # if qm==4:
              #     print(f'qm={qm}, tm={tm}, score={x}')
              if x > max:
                max = x
                maxmeasure = tm
              if tm <= 16 and x > max8:
                max8 = x
                maxmeasure8 = tm
            #print(maxbeat, x)
            score += max
            #print(max, maxmeasure)
            for i in range(length):
              pattern.append(maxmeasure+i)
              pattern8.append(maxmeasure8+i)
          gscore += score
          patterns.append((score, pattern, pattern8))

        if not np.isnan(gscore):
          #scores.append((gscore, track, pitch, patterns[0][0], patterns[0][1], patterns[0][2], patterns[1][0], patterns[1][1], patterns[1][2], patterns[2][0], patterns[2][1], patterns[2][2], patterns[3][0], patterns[3][1], patterns[3][2]))
          scores.append((gscore, track, pitch, patterns[0][0], patterns[0][1], patterns[0][2], patterns[1][0], patterns[1][1], patterns[1][2]))
                        #patterns[2][0], patterns[2][1], patterns[2][2], patterns[3][0], patterns[3][1], patterns[3][2]))
        else:
          print('isnan')
        tstart+=int(BEATS*divisionsPerBeat)
  return scores

In [ ]:
def splitLine(line):
  return int(line.strip().split(':')[1])

In [ ]:
trackcat = dict()
for path in tqdm(list(Path('/mnt/c/Users/andre/OneDrive/Music/Library/').rglob('*.mp3'))):
  #path = '/mnt/c/Users/andre/OneDrive/Music/Library/0KVU9o7yhQHIcXtCdYmM4IX/Alice DJ - Better Off Alone (Radio Edit).mp3'
  dumpfile = Path(f'{str(path)}-.spleeter')

  file1 = open(dumpfile, 'r')
  lines = file1.readlines()

  bass = drums = other = vocals = 0

  bass = splitLine(lines[0])
  drums = splitLine(lines[1])
  other = splitLine(lines[2])
  vocals = splitLine(lines[3])

  print(dumpfile, bass, drums, other, vocals)

  trackcat[str(path)]=(bass, drums, other, vocals)

In [ ]:
def analyzeSpleeter(tc):
  bass, drums, other, vocals = tc
  z = min(tc)
  sps = 0
  spe = 4
  if vocals > z:
    spe = vocals
    if vocals % 2 == 1:
      spe = vocals+1

    sps = max(0, spe-4)

  return sps, spe


In [ ]:
analyzeSpleeter(trackcat[tb])

In [ ]:
qchroma_ = qmanual
q0 = qchroma_
q1 = np.roll(qchroma_, 1, axis=1)
q2 = np.roll(qchroma_, 2, axis=1)

In [ ]:
q0.shape

In [ ]:
q1.shape

In [ ]:
st = np.concatenate((q0, q0, q0, q0), axis=0)

In [ ]:
st.shape

###ultraWideQSpleeterPitchMatch

In [ ]:
#EX ULTRAWIDE Q, with 2 Measures match
def ultraWideQSpleeterPitchMatch(trackse, trackme, qchroma_, divisionsPerBeat=1):
  dpb = divisionsPerBeat
  scores = list()
  qbeats = int(qchroma_.shape[0]//divisionsPerBeat)
  qmeasures = qbeats // 4

  print('qbeats:=', qbeats, '; qmeasures:=', qmeasures)

  NUMTRACKS=len(trackse)

  for w in tqdm(range(0,NUMTRACKS,500)):  
    wend = min(w+500, NUMTRACKS)
    print(w,wend)
    q0 = qchroma_
    q1 = np.roll(qchroma_, 1, axis=1)
    q2 = np.roll(qchroma_, 2, axis=1)
    q3 = np.roll(qchroma_, 3, axis=1)
    qm1 = np.roll(qchroma_, -1, axis=1)
    qm2 = np.roll(qchroma_, -2, axis=1)
    qm3 = np.roll(qchroma_, -3, axis=1)
    qr = np.concatenate((q0, q1, q2, q3, qm1, qm2, qm3), axis=0)
    s = np.matmul(trackme[w*int(BEATS*divisionsPerBeat):wend*int(BEATS*divisionsPerBeat),:], qr.T)
    #print(trackme.shape, qr.shape, s.shape)
    tstart = 0
    for ti, track in enumerate(list(trackse)[w:wend]):
      #spleetes start/end
      tc = trackcat[track]
      sps, spe = analyzeSpleeter(tc)
      length = spe-sps
      score = 0
      scorenp = 0
      pattern = list()
      pitches = list()
      pitch = 0
      for qm in range(0,qmeasures, 1):
        tm = sps + qm % length
        #print(f'Coordinates {tm}/{qm}')
        x0  = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+1, dpb), m2i(qm, dpb):m2i(qm+1, dpb)])
        x1  = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+1, dpb), m2i(qm+1*qmeasures, dpb):m2i(qm+1*qmeasures+1, dpb)])
        x2  = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+1, dpb), m2i(qm+2*qmeasures, dpb):m2i(qm+2*qmeasures+1, dpb)])
        x3  = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+1, dpb), m2i(qm+3*qmeasures, dpb):m2i(qm+3*qmeasures+1, dpb)])
        xm1 = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+1, dpb), m2i(qm+4*qmeasures, dpb):m2i(qm+4*qmeasures+1, dpb)])
        xm2 = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+1, dpb), m2i(qm+5*qmeasures, dpb):m2i(qm+5*qmeasures+1, dpb)])
        xm3 = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+1, dpb), m2i(qm+6*qmeasures, dpb):m2i(qm+6*qmeasures+1, dpb)])
        
        pitchedx = [x0, x1, x2, x3, xm1, xm2, xm3]
        maxp = max(pitchedx)
        maxi = pitchedx.index(maxp)
        #print(pitchedx, maxp, maxi)

        pitch = maxi
        # if maxi > 3:
        #   pitch = 3-maxi
        pitches.append(pitch)

 
        score += maxp
        scorenp += x0
      for i in range(qmeasures):
        pattern.append(sps+i % length)

      if not np.isnan(score):
        scores.append((score, track, pitch, pattern, pitches))
      else:
        print('isnan')
      tstart+=int(BEATS*divisionsPerBeat)
  return scores

###ultraWideQSpleeterMatch

In [ ]:
#EX ULTRAWIDE Q, with 2 Measures match
def ultraWideQSpleeterMatch(trackse, trackme, qchroma_, divisionsPerBeat=1):
  dpb = divisionsPerBeat
  scores = list()
  qbeats = int(qchroma_.shape[0]//divisionsPerBeat)
  qmeasures = qbeats // 4

  print('qbeats:=', qbeats, '; qmeasures:=', qmeasures)

  NUMTRACKS=len(trackse)

  for w in tqdm(range(0,NUMTRACKS,500)):  
    for pitch in tqdm([-3, -2, -1, 0, 1, 2, 3]):
      wend = min(w+500, NUMTRACKS)
      print(w,wend)
      qr = np.roll(qchroma_, pitch, axis=1)
      s = np.matmul(trackme[w*int(BEATS*divisionsPerBeat):wend*int(BEATS*divisionsPerBeat),:], qr.T)
      #print(trackme.shape, qr.shape, s.shape)
      tstart = 0
      for ti, track in enumerate(list(trackse)[w:wend]):
        #spleetes start/end
        tc = trackcat[track]
        sps, spe = analyzeSpleeter(tc)
        length = spe-sps
        score = 0
        pattern = list()
        for qm in range(0,qmeasures, length):
          x = np.trace(s[tstart+m2i(sps, dpb):tstart+m2i(sps+length, dpb), m2i(qm, dpb):m2i(qm+length, dpb)])
          score += x
        for i in range(qmeasures):
          pattern.append(sps+i % length)

        if not np.isnan(score):
          scores.append((score, track, pitch, pattern))
        else:
          print('isnan')
        tstart+=int(BEATS*divisionsPerBeat)
  return scores

In [ ]:
#EX ULTRAWIDE Q, with 2 Measures match
def ultraWideQSpleeterMatch1Bar(trackse, trackme, qchroma_, divisionsPerBeat=1):
  dpb = divisionsPerBeat
  scores = list()
  qbeats = int(qchroma_.shape[0]//divisionsPerBeat)
  qmeasures = qbeats // 4

  print('qbeats:=', qbeats, '; qmeasures:=', qmeasures)

  NUMTRACKS=len(trackse)

  for w in tqdm(range(0,NUMTRACKS,500)):  
    for pitch in tqdm([-3, -2, -1, 0, 1, 2, 3]):
      wend = min(w+500, NUMTRACKS)
      print(w,wend)
      qr = np.roll(qchroma_, pitch, axis=1)
      s = np.matmul(trackme[w*int(BEATS*divisionsPerBeat):wend*int(BEATS*divisionsPerBeat),:], qr.T)
      #print(trackme.shape, qr.shape, s.shape)
      tstart = 0
      for ti, track in enumerate(list(trackse)[w:wend]):
        #spleetes start/end
        tc = trackcat[track]
        sps, spe = analyzeSpleeter(tc)
        length = spe-sps
        score = 0
        pattern = list()
        for qm in range(0,qmeasures, 1):
          maxx=-1.0
          maxtmx = -1
          for tm in range(sps, spe):
            length=1
            x = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+length, dpb), m2i(qm, dpb):m2i(qm+length, dpb)])
            if x > maxx:
              maxx = x
              maxtm = tm
          score += maxx
          pattern.append(maxtm)

        if not np.isnan(score):
          scores.append((score, track, pitch, pattern))
        else:
          print('isnan')
        tstart+=int(BEATS*divisionsPerBeat)
  return scores

In [ ]:
findTrack('girls')

In [ ]:
trackcat[tb]

In [ ]:
qs_

In [ ]:
trackse = trackm.keys()
tb = '/mnt/c/Users/andre/OneDrive/Music/Library/DB/Baltimora - Tarzan Boy.mp3'
gjwhf = '/mnt/c/Users/andre/OneDrive/Music/Library/6zqADRDfaIwfcBJQgTnLKo/Cyndi Lauper - Girls Just Want to Have Fun.mp3'
bty = '/mnt/c/Users/andre/OneDrive/Music/Library/DB/Boytronic - You.mp3'
#trackse = [bty]
#trackse = [tb, gjwhf]
#trackse = [str(qt)]
valuese = [trackm[k] for k in trackse]
trackme = np.concatenate(valuese)

scoresfm = ultraWideQSpleeterPitchMatch(trackse, trackme, trackf[str(qt)][48:112], divisionsPerBeat=1) #qmix=0.63  trackf[tb][64:128] / trackm[str(qt)][80:144]

In [ ]:
#trackcat[gjwhf]

In [ ]:
print(scoresfm[0])

In [ ]:
scoresfm.sort(key=lambda s: s[0], reverse=True)

In [ ]:
print(scoresfm[0])
print(scoresfm[1])
print(scoresfm[2])
print(scoresfm[3])
print(scoresfm[4])

In [ ]:
6 in [1, 2, 3]

In [ ]:
seen = dict()
pos = 0

In [ ]:
loop=loops[0]
shown = 0


while True:
 


  s = scoresfm[pos]
  pos += 1
  print(s)

  id = s[1]

  if id in seen:
    count = seen[id]
    if count > 0:
      continue
    else:
      count +=1
      seen[id]=count
  else:
    seen[id]=1

  # if s[2] in [4, 6, 8]:
  #   print(lookup[id].attrib['Comments'])  
  #   outputMix(loop, s, scorest[sind])
  #   outputMix(loop, s, scorest[sind], _2nd=True)

    #shown+=1
    # if shown < 100:
    #   continue


  print(getComment(lookup(id)))  
  print(trackcat[id])
  # if s[1]=='/mnt/c/Users/andre/OneDrive/Music/Library/DB/Alphaville - Sounds Like a Melody (2019 Remaster).mp3':
  #   continue
    
  mix = outputMixPitch(loop, s, 'Manual4', _pattternind=3, showmix=True)
  #mixes.append(mix)
  # outputMix(loop, s, '4', _pattternind=7)
  # outputMix(loop, s, '2<8', _pattternind=11)
  # outputMix(loop, s, '1<8', _pattternind=14)
  shown+=1

  if shown > 20:
    # mixout = AudioSegment.empty()
    # for mix in mixes:
    #   mixout = mixout + mix
    # IPython.display.display(mixout)
    break
    

In [ ]:

#EX ULTRAWIDE Q, with 2 Measures match
def ultraWideQ2MeasuresMatch2(trackse, trackme, qchroma_, qchroma1_, qmix=0.5, divisionsPerBeat=1):
  dpb = divisionsPerBeat
  scores = list()
  qbeats = qchroma_.shape[0]//divisionsPerBeat
  qmeasures = qbeats // 4

  print('qbeats=', qbeats, '; qmeasures=', qmeasures)

  NUMTRACKS=len(trackse)

  for w in tqdm(range(0,NUMTRACKS,500)):  
    for pitch in tqdm([9, 10, 11, 0, 1, 2, 3]):
      wend = min(w+500, NUMTRACKS)
      print(w,wend)
      qr = np.roll(qchroma_, pitch, axis=1)
      qr1 = np.roll(qchroma1_, pitch, axis=1)
      s = np.matmul(trackme[w*BEATS*divisionsPerBeat:wend*BEATS*divisionsPerBeat,:], qr.T)
      s1 = np.matmul(trackme[w*BEATS*divisionsPerBeat:wend*BEATS*divisionsPerBeat,:], qr1.T)
      print(trackme.shape, qr.shape, s.shape)
      tstart = 0
      xavg = 0.
      xcount=0
      x1avg=0.
      for ti, track in enumerate(list(trackse)[w:wend]):
        #print('********************* New Track ************')
        gscore = 0
        patterns = list()
        pattern8s = list()
        for length in [16, 8]:
          score = 0
          pattern = list()
          pattern8 = list()
          for qm in range(0,qmeasures, length):
            max         = -1.0
            maxmeasure  = -1
            max8        = -1.0
            maxmeasure8 = -1
            for tm in range(0,BEATS//4-length+1,4): #ACHTUNG: nicht lenght, sondern sliding TM
              x = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+length, dpb), m2i(qm, dpb):m2i(qm+length, dpb)])
              x1 = np.trace(s1[tstart+m2i(tm, dpb):tstart+m2i(tm+length, dpb), m2i(qm, dpb):m2i(qm+length, dpb)])
              xcount+=1
              xavg += (x - xavg)/xcount
              x1avg += (x1 - x1avg)/xcount
              x = x*qmix+x1*(1-qmix)

              # if tm==0:
              #   print(ti, track, pitch, 'qm',qm, 'tm',tm, 'tstart', tstart, x)
              #print(tm, qm, x, m2i(tm, dpb), m2i(qm, dpb))
              # if qm==4:
              #     print(f'qm={qm}, tm={tm}, score={x}')
              if x > max:
                max = x
                maxmeasure = tm
              if tm <= 16 and x > max8:
                max8 = x
                maxmeasure8 = tm
            #print(maxbeat, x)
            score += max
            #print(max, maxmeasure)
            # if maxmeasure % 2 != 0:
            #   print(track)
            for i in range(length):
              pattern.append(maxmeasure+i)
              pattern8.append(maxmeasure8+i)
          gscore += score
          patterns.append((score, pattern, pattern8))

        if not np.isnan(gscore):
          #print(xavg, x1avg)
          #scores.append((gscore, track, pitch, patterns[0][0], patterns[0][1], patterns[0][2], patterns[1][0], patterns[1][1], patterns[1][2], patterns[2][0], patterns[2][1], patterns[2][2], patterns[3][0], patterns[3][1], patterns[3][2]))
          scores.append((gscore, track, pitch, patterns[0][0], patterns[0][1], patterns[0][2], patterns[1][0], patterns[1][1], patterns[1][2], xavg/x1avg))
                        #patterns[2][0], patterns[2][1], patterns[2][2], patterns[3][0], patterns[3][1], patterns[3][2]))
        else:
          print('isnan')
        tstart+=BEATS*divisionsPerBeat
  return scores

###ultraWideQ2MeasuresPatternMatch

In [ ]:
def pattern2ind(pattern):
  print(pattern)
  pattern2ind = dict()
  entries = list(set(pattern))
  for e in entries:
    indices = [i for i, x in enumerate(pattern) if x == e]
    pattern2ind[e]=indices
  return pattern2ind

In [ ]:
pattern2ind(['A', 'A', 'B', 'B', 'A', 'B', 'C'])

In [ ]:
#EX ULTRAWIDE Q, with 2 Measures match
def ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma_, pattern, divisionsPerBeat=1):
  patternind = pattern2ind(pattern)
  dpb = divisionsPerBeat
  scores = list()
  qbeats = qchroma_.shape[0]//divisionsPerBeat
  qmeasures = qbeats // 4
  print('qbeats=', qbeats, '; qmeasures=', qmeasures, '; pattern=', len(pattern)*4, '; OK=', qmeasures==len(pattern)*4)
  if qmeasures!=len(pattern)*4:
    return

  NUMTRACKS=len(trackse)




  for w in tqdm(range(0,NUMTRACKS,500)):

    for pitch in tqdm(9, 10, 11, 0, 1, 2, 3):
      wend = min(w+500, NUMTRACKS)
      qr = np.roll(qchroma_, pitch, axis=1)
      print(w,wend)
      s = np.matmul(trackme[w*96*divisionsPerBeat:wend*96*divisionsPerBeat,:], qr.T)
      print(trackme.shape, qr.shape, s.shape)
      tstart = 0
      for track in list(trackse)[w:wend]:
        # gscore = 0
        # xpatterns = list()
        # xpattern8s = list()
        length = 4
        score = 0
        # xpattern = list()
        # xpattern8 = list()
        symbolmeasure =  dict()
        for patternsymbol in patternind.keys():
            patternindices = patternind[patternsymbol]
            maxtmscore = -1.0
            maxtm      = -1
            for tm in range(0, 24-length+1, length):
              tmscore = 0
              for pi in patternindices:
                #Pro pattern index = 4 Bars
                qm = pi*4
                x = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+length, dpb), m2i(qm, dpb):m2i(qm+length, dpb)])
                tmscore += x
                # if qm==4:
                #   print(f'qm={qm}, tm={tm}, score={tmscore}')
              if tmscore > maxtmscore:
                maxtmscore = tmscore
                maxtm = tm
            score += maxtmscore
            symbolmeasure[patternsymbol]=maxtm
            
        if not np.isnan(score):
          scores.append((score, track, pitch, ind2pattern(symbolmeasure, pattern, length), symbolmeasure))
        else:
          print('isnan')
        tstart+=96*divisionsPerBeat
  return scores

TESTING Infrastructure

In [ ]:
# trackse = ['/mnt/c/Users/andre/OneDrive/Music/Library/DB/Pet Shop Boys - Always on My Mind.mp3']
# valuese = [trackf[k] for k in trackse]
# trackme = np.concatenate(valuese)

# print('MATCH')
# scoresfm1 = ultraWideQ2MeasuresMatch(trackse, trackme, qchromas[0][0],  divisionsPerBeat=1)
# print('PATTERN')
# scoresfm2 = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchromas[0][0], pattern=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],  divisionsPerBeat=1)



In [ ]:
# trackse = trackf.keys()
# valuese = [trackf[k] for k in trackse]
# trackme = np.concatenate(valuese)

# #scoresfm = ultraWideQ2MeasuresMatch(list(trackse)[0:3], trackme[0:3*96*4*4], qchromas[0][0], divisionsPerBeat=4)
# scoresfm = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchromas[0][0], pattern=['A', 'A', 'B', 'B', 'C', 'D', 'C', 'D'],  divisionsPerBeat=1)

In [ ]:
# trackse = trackf.keys()
# valuese = [trackf[k] for k in trackse]
# trackme = np.concatenate(valuese)

# #scoresfm = ultraWideQ2MeasuresMatch(list(trackse)[0:3], trackme[0:3*96*4*4], qchromas[0][0], divisionsPerBeat=4)
# scoresfm = ultraWideQ2MeasuresMatch(trackse, trackme, qchromas[0][0], divisionsPerBeat=1)


In [ ]:
# scoresfm.sort(key=lambda s: s[0], reverse=True)

In [ ]:
# print(scoresfm[0])

In [ ]:
# #verbessern: zuest anzeige, dann musik
# loop = loops[0]
# for s in scoresfm[220:230]:
#   if s[2] in [4,6,8] or True:
#     print(s)
#     outputMix(loop, s, '8', _pattternind=3)
#     # outputMix(loop, s, '4', _pattternind=7)
#     # outputMix(loop, s, '2', _pattternind=10)
#     # outputMix(loop, s, '1', _pattternind=13)


In [ ]:
def getScores2m(qchroma, divisionperbeat=1, track=None):

  print('Beats=', qchroma[0].shape)

  if track is None:
    trackse = trackf.keys()
  else:
    trackse = [str(track)]
  valuese = [trackf[k] for k in trackse]
  trackme = np.concatenate(valuese)

  scoresfm = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[0], divisionsPerBeat=divisionperbeat)
  scoresff = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[1], divisionsPerBeat=divisionperbeat)
  #scoresfk = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[2], divisionsPerBeat=divisionperbeat)



  if track is None:
    trackse = trackm.keys()
  else:
    trackse = [str(track)]
  valuese = [trackm[k] for k in trackse]
  trackme = np.concatenate(valuese)

  scoresmm = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[0], divisionsPerBeat=divisionperbeat)
  scoresmf = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[1], divisionsPerBeat=divisionperbeat)
  # scoresmk = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[2], divisionsPerBeat=divisionperbeat)

  # if track is None:
  #   trackse = trackp.keys()
  # else:
  #   trackse = [str(track)]
  # valuese = [trackp[k] for k in trackse]
  # trackme = np.concatenate(valuese)

  # scoreskm = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[0], divisionsPerBeat=divisionperbeat)
  # scoreskf = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[1], divisionsPerBeat=divisionperbeat)
  # scoreskk = ultraWideQ2MeasuresMatch(trackse, trackme, qchroma[2], divisionsPerBeat=divisionperbeat)
 
  
  scoresfm.sort(key=lambda s: s[0], reverse=True)
  scoresff.sort(key=lambda s: s[0], reverse=True)
  # scoresfk.sort(key=lambda s: s[0], reverse=True)
  scoresmm.sort(key=lambda s: s[0], reverse=True)
  scoresmf.sort(key=lambda s: s[0], reverse=True)
  # scoresmk.sort(key=lambda s: s[0], reverse=True)
  # scoreskm.sort(key=lambda s: s[0], reverse=True)
  # scoreskf.sort(key=lambda s: s[0], reverse=True)
  # scoreskk.sort(key=lambda s: s[0], reverse=True)

  scoresl = [None] * 4
  scoresl[0] = scoresmm
  scoresl[1] = scoresff
  #scoresl[2] = list() #scoreskk

  scoresl[2] = scoresmf
  #scoresl[4] = list() #scoresmk

  scoresl[3] = scoresfm
  #scoresl[6] = list() #scoresfk

  #scoresl[7] = list() #scoreskm
  #scoresl[8] = list() #scoreskf

  #scorest = ['smm', 'sff', 'skk', 'smf', 'smk', 'sfm', 'sfk', 'skm', 'skf']
  scorest = ['smm', 'sff', 'smf', 'sfm']


  return scoresl, scorest


In [ ]:
def getScores2mPattern(qchroma, pattern, divisionperbeat=1, track=None):

  print('Beats=', qchroma[0].shape)

  if track is None:
    trackse = trackf.keys()
  else:
    trackse = [str(track)]
  valuese = [trackf[k] for k in trackse]
  trackme = np.concatenate(valuese)

  scoresfm = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[0], pattern, divisionsPerBeat=divisionperbeat)
  scoresff = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[1], pattern, divisionsPerBeat=divisionperbeat)
  scoresfk = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[2], pattern, divisionsPerBeat=divisionperbeat)


  if track is None:
    trackse = trackm.keys()
  else:
    trackse = [str(track)]
  valuese = [trackm[k] for k in trackse]
  trackme = np.concatenate(valuese)

  scoresmm = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[0], pattern, divisionsPerBeat=divisionperbeat)
  scoresmf = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[1], pattern, divisionsPerBeat=divisionperbeat)
  scoresmk = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[2], pattern, divisionsPerBeat=divisionperbeat)


  if track is None:
    trackse = trackp.keys()
  else:
    trackse = [str(track)]
  valuese = [trackp[k] for k in trackse]
  trackme = np.concatenate(valuese)

  scoreskm = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[0], pattern, divisionsPerBeat=divisionperbeat)
  scoreskf = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[1], pattern, divisionsPerBeat=divisionperbeat)
  scoreskk = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchroma[2], pattern, divisionsPerBeat=divisionperbeat)
 
  
  scoresfm.sort(key=lambda s: s[0], reverse=True)
  scoresff.sort(key=lambda s: s[0], reverse=True)
  scoresfk.sort(key=lambda s: s[0], reverse=True)
  scoresmm.sort(key=lambda s: s[0], reverse=True)
  scoresmf.sort(key=lambda s: s[0], reverse=True)
  scoresmk.sort(key=lambda s: s[0], reverse=True)
  scoreskm.sort(key=lambda s: s[0], reverse=True)
  scoreskf.sort(key=lambda s: s[0], reverse=True)
  scoreskk.sort(key=lambda s: s[0], reverse=True)

  scoresl = [None] * 9
  scoresl[0] = scoresmm
  scoresl[1] = scoresff
  scoresl[2] = scoreskk

  scoresl[3] = scoresmf
  scoresl[4] = scoresmk

  scoresl[5] = scoresfm
  scoresl[6] = scoresfk

  scoresl[7] = scoreskm
  scoresl[8] = scoreskf

  scorest = ['smm', 'sff', 'skk', 'smf', 'smk', 'sfm', 'sfk', 'skm', 'skf']


  return scoresl, scorest


##Minichroma

In [ ]:
#minichromas = (qchromas[0][0][0:4*16], qchromas[0][1][0:4*16], qchromas[0][2][0:4*16])

In [ ]:
#scoresl, scorest = getScores(minichromas, divisionperbeat=4) #chroma below

In [ ]:
#scoresl, scorest = getScoresPattern(qchromas[0],  divisionperbeat=4)

In [ ]:
scoresl, scorest = getScores2m(qchromas[0],  divisionperbeat=1)

In [ ]:
scoresl, scorest = getScores2mPattern(qchromas[0],  pattern=['A', 'A', 'A', 'A', 'B', 'C'], divisionperbeat=1)

In [ ]:
seen = dict()
pos = 0

In [ ]:
loop=loops[0]
shown = 0

mixes = list()

while True:
  sind = pos % 4
  spos = pos // 4
  print(sind, spos)


  #s = scoresl[0][pos]
  pos += 1


  s = scoresl[sind][spos]
  print(scorest[sind], s)

  id = s[1]

  if id in seen:
    count = seen[id]
    if count > 0:
      continue
    else:
      count +=1
      seen[id]=count
  else:
    seen[id]=1

  # if s[2] in [4, 6, 8]:
  #   print(lookup[id].attrib['Comments'])  
  #   outputMix(loop, s, scorest[sind])
  #   outputMix(loop, s, scorest[sind], _2nd=True)

    #shown+=1
    # if shown < 100:
    #   continue


  print(getComment(lookup(id)))  
  # if s[1]=='/mnt/c/Users/andre/OneDrive/Music/Library/DB/Alphaville - Sounds Like a Melody (2019 Remaster).mp3':
  #   continue
    
  mix = outputMix(loop, s, '8', _pattternind=4, showmix=True)
  #mixes.append(mix)
  # outputMix(loop, s, '4', _pattternind=7)
  # outputMix(loop, s, '2<8', _pattternind=11)
  # outputMix(loop, s, '1<8', _pattternind=14)
  shown+=1

  if shown > 20:
    # mixout = AudioSegment.empty()
    # for mix in mixes:
    #   mixout = mixout + mix
    # IPython.display.display(mixout)
    break
    

In [ ]:
exportBeatgridVDJ(qt)

In [ ]:
exportCueBeats(qt, [qstart, qend])

In [ ]:
oscore = (48.38206993613574, '/mnt/c/Users/andre/OneDrive/Music/Library/DB/Peter Gabriel - Sledgehammer.mp3', 6, [12, 13, 14, 15, 12, 13, 14, 15, 12, 13, 14, 15, 12, 13, 14, 15, 8, 9, 10, 11, 4, 5, 6, 7], {'C': 4, 'A': 12, 'B': 8})
ot = oscore[1]
exportBeatgridVDJ(ot)

In [ ]:
exportCueMeasures(ot, [4,8,12])

In [ ]:
rmme = [
'/mnt/c/Users/andre/OneDrive/Music/Library/DB/Max Werner - Rain In May.beats',
'/mnt/c/Users/andre/OneDrive/Music/Library/5fevftlD8al7MLhmBE9bQfX/The Cars - Drive.beats'
]

for f in rmme:
  print(f'rm {f}')
  print(f'rm {f.replace(".beats", ".mp3-pair.m4116data")}')

In [ ]:
def getAlternatePatterns(t, l):
  scoresl, scorest = getScores2m(qchromas[0], divisionperbeat=1, track=l[1])
  for i, z in enumerate(scorest):
    if z==t:
      s = scoresl[i][0]
      print(l[3])
      print(s[4])
      outputMix(loop, s, '8', _pattternind=4, showmix=True)
      print(s[7])
      outputMix(loop, s, '8', _pattternind=7, showmix=True)

In [ ]:
t, l = 'sff', (109.00055980682373, "/mnt/c/Users/andre/OneDrive/Music/Library/DB/Bronski Beat - Why _ (12'' Version; 2008 Remastered Version).mp3", 7, [3, 4, 5, 6, 3, 4, 5, 6, 3, 4, 5, 6, 3, 4, 5, 6, 1, 2, 3, 4, 3, 4, 5, 6, 1, 2, 3, 4, 3, 4, 5, 6], {'A': 3, 'B': 3, 'D': 3, 'C': 1})
getAlternatePatterns(t, l)


In [ ]:
def getAlternatePatternPatterns(t, l):
  scoresl, scorest = getScores2mPattern(qchromas[0],  pattern=['A', 'A', 'A', 'A', 'B', 'C'], divisionperbeat=1, track=l[1])
  for i, z in enumerate(scorest):
    if z==t or True:
      s = scoresl[i][0]
      print(z, s[3])
      outputMix(loop, s, '4P', _pattternind=3, showmix=True)

In [ ]:
t, l = 'sfm', (48.38206993613574, '/mnt/c/Users/andre/OneDrive/Music/Library/DB/Peter Gabriel - Sledgehammer.mp3', 6, [12, 13, 14, 15, 12, 13, 14, 15, 12, 13, 14, 15, 12, 13, 14, 15, 8, 9, 10, 11, 4, 5, 6, 7], {'C': 4, 'A': 12, 'B': 8})
getAlternatePatternPatterns(t, l)




##Real MANUAL MODE

In [ ]:
loop = loops[0]

In [ ]:
#Manual

#outputMix(loop, smanual, '*manual*', loopdb=-12, ovldb=-9, filename='karma-faith-01.wav', 4)#, hps=1000)


smanual = (146.08952842862615, '/mnt/c/Users/andre/OneDrive/Music/Library/DB/Fancy - Chinese Eyes (12_ Version).mp3', 0, 35.176534711062395, [12, 13, 14, 15, 16, 17, 18, 19, 4, 5, 6, 7, 8, 9, 10, 11], [4, 5, 6, 7, 8, 9, 10, 11, 4, 5, 6, 7, 8, 9, 10, 11], 35.22484599441405, [4, 5, 6, 7, 9, 10, 11, 12, 8, 9, 10, 11, 8, 9, 10, 11], [4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7], 37.522901427996985, [8, 9, 14, 15, 13, 14, 9, 10, 8, 9, 5, 6, 8, 9, 9, 10], [4, 5, 6, 7, 0, 1, 5, 6, 4, 5, 5, 6, 4, 5, 5, 6], 38.16524629515273, [4, 9, 11, 14, 13, 1, 9, 14, 9, 9, 5, 14, 9, 9, 5, 10], [4, 5, 1, 6, 4, 1, 4, 0, 4, 5, 5, 6, 4, 5, 5, 6])

smanual =(44.40090377751787, '/mnt/c/Users/andre/OneDrive/Music/Library/DB/The Twins - Ballet Dancer.mp3', 2, 22.200451888758934, [32, 33, 34, 35, 32, 33, 34, 35], [8, 9, 10, 11, 12, 13, 14, 15], 22.200451888758934, [32, 33, 34, 35, 32, 33, 34, 35], [16, 17, 18, 19, 0, 1, 2, 3], 0.6709413086989743)


smanual = (75.41366288169134, '/mnt/c/Users/andre/OneDrive/Music/Library/0KVU9o7yhQHIcXtCdYmM4IX/Die Ärzte - Westerland.mp3', 9, 37.10934652306713, [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], 38.304316358624206, [20, 21, 22, 23, 24, 25, 26, 27, 20, 21, 22, 23, 24, 25, 26, 27], [8, 9, 10, 11, 12, 13, 14, 15, 8, 9, 10, 11, 12, 13, 14, 15], 0.5950788286839958)


smanual = (35.6648821932376, '/mnt/c/Users/andre/OneDrive/Music/Library/2hDNJUa6EqJLfWmChadlvRX/The J. Geils Band - Centerfold.mp3', -3, [4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7])
none = outputMix(loop, smanual, '*manual*', loopdb=-12, ovldb=-15, _pattternind=3) #7


##Studio - Chop & Export

In [ ]:
def plotTransition(audio):
  plt.figure(figsize=(20,10))  #plt.subplots(len(channels), sharex=True, figsize=(50,100))
  #fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')


  channel_sounds = audio.split_to_mono()
  samples = [s.get_array_of_samples() for s in channel_sounds]

  fp_arr = np.array(samples).T.astype(np.float32)
  fp_arr /= np.iinfo(samples[0].typecode).max

  print(fp_arr.shape)
  plt.plot(fp_arr)
  plt.axvline(x=fp_arr.shape[0]/2)
  plt.show()

In [ ]:
def chopAndExport(trackid, pattern):

  track = lookup[trackid]

  bg = BeatGrid(track)
  firstbeat = bg.get111ind()
  
  audio = AudioSegment.from_file(MIXX / getFilename(track), format="mp3")
  
  for i, (code, m) in enumerate(pattern.items()):
    print(i, code, m)

    start         = bg.getBeat_p(firstbeat+m*4)
    end           = bg.getBeat_p(firstbeat+m*4+16)
    bpm           = computeBPM_s(start, end, beats=16)
    traktorend    = bg.getBeat_p(firstbeat+m*4+16+4)


    start_ms = int(start*1000)-5
    end_ms   = int(end*1000)-5
    
    audio[start_ms:end_ms].export(f'{trackid}-m{m:02}-{code}-{bpm:.3f}.wav', format='wav')
    plotTransition(audio[end_ms-100:end_ms+100])

In [ ]:
ID = 'C:/:Users/:andre/:OneDrive/:Music/:Library/:DB/:ABBA - Super Trouper.mp3'

In [ ]:
tt = lookup(ID)

In [ ]:
taudio = AudioSegment.from_file(getWSLFilename(tt), format="mp3")
tmis = AudioSegment.from_file('karma-trooper-01.wav', format="wav")

In [ ]:
tbg =   BeatGrid(tt)

In [ ]:
start  = tbg.getBeat_p(tbg.get111ind()+8*4)
end  = tbg.getBeat_p(tbg.get111ind()+8*4+32)

In [ ]:
(tmis+(taudio[int(start*1000):int(end*1000)]-15)).export('karma-trooper-long.wav', format="wav")

In [ ]:
lookup[s_ovl[1]].attrib

In [ ]:
loops[0][0].export(f'{TrackID}-q-{loops[0][1]:.3f}.wav', format='wav')

In [ ]:
s_ovl[5]

In [ ]:
outputMix(loops[0], s_ovl, "save", loopdb=-15, ovldb=-12, filename='mixout.wav', hps=None, _2nd=False)

In [ ]:
#sfm (99.7968116132125, '6918', 7, [5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8], [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4], {'C': 5, 'V1': 0, 'V2': 0}, {'C': 1, 'V1': 0, 'V2': 0})

#chopAndExport('6918',  {'C': 5, 'V1': 0, 'V2': 0})
print(f'Pitch={s_ovl[2]}')
#chopAndExport(s_ovl[1],  s_ovl[5])

manualp = {'0': 0, '2': 2, '4': 4, '6': 6, '8': 8, '10': 10, '12': 12, '14': 14, '16': 16, '18': 18, '20': 20, '22': 22}
chopAndExport(s_ovl[1],  manualp)


##Full Song Mode

In [ ]:
ofullsong =  (77.00000000000003, '/mnt/c/Users/andre/OneDrive/Music/Library/DB/Prince - Gold.mp3', 10, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3], {'C': 0, 'D': 0, 'B': 0, 'A': 0})


In [ ]:
ot = ofullsong[1]

In [ ]:
okey = getMixedInKeyKey(ot)
okey = okey.split('/')[0] #C#/D#
okey = okey.strip()
print(ot, '>'+okey+'<')

In [ ]:
opcps, ox = createPCPs(okey)
ox

In [ ]:
obg = BeatGrid.fromTrack(ot)

In [ ]:
ostart = obg.get111ind()
oend   = obg.getLastBeatInd()
print(ostart, oend)
obg.beats[ostart], obg.beats[oend], (oend-ostart)/4

In [ ]:
oaudio = AudioSegment.from_file(ot, format="mp3")

In [ ]:
o, sr = librosa.load(str(ot)) 

In [ ]:
bt1     = obg.getDownBeatTimes(1)
bt2     = obg.getDownBeatTimes(2)
bt3     = obg.getDownBeatTimes(3)
bt4     = obg.getDownBeatTimes(4)


#print(dbs_ind)
b1clicks = librosa.clicks(bt1, sr=sr, click_freq=1000, length=len(o))
b2clicks = librosa.clicks(bt2, sr=sr, click_freq=1500, length=len(o))
b3clicks = librosa.clicks(bt3, sr=sr, click_freq=2000, length=len(o))
b4clicks = librosa.clicks(bt4, sr=sr, click_freq=2500, length=len(o))

clicks = b1clicks + b2clicks + b3clicks + b4clicks
#clicks = b1clicks



#dbclicks = librosa.clicks(mbg.getDownBeatTimes(3), sr=sr, click_freq=2000, length=len(x))

sf.write('oplusclicks.wav', o+clicks, sr, subtype='PCM_24')

clickseg = AudioSegment.from_wav('oplusclicks.wav')
clickseg

In [ ]:
#m1

In [ ]:
list(enumerate(obg.beats)) #73.49,   -62

In [ ]:
#(134, (75.72, 1)),
406/4

In [ ]:
#obeats = ochromas[0][0].shape[0]//1
#omeasures = obeats // 4
olist = [(1,9)] 


In [ ]:
os_ = list()
fb = obg.get111ind()
for oo in olist:
  os_.append((fb+oo[0]*4, fb+oo[1]*4))



In [ ]:
oloops, _ = chopLoops(oaudio, obg, os_)

In [ ]:
oloops[0][0]

In [ ]:
ochromas = list()
ostart = obg.get111ind()
oend   = obg.getLastBeatInd()
_33beats = obg.getBeatTimes()[ostart:oend+1]
print(len(_33beats))

omedians4, omedians1, omedians16 = processQueryLoop(o, _33beats)

ochroma = np.stack(omedians4)
ofree   = matchPCPS(ochroma,freepcps)
okey    = matchPCPS(ochroma,qpcps)
ochromas.append((ochroma, ofree, okey))

In [ ]:
ochromas[0][0].shape

In [ ]:
#measureToIndex
def m2i(m, divisionsPerBeat):
  return m*4*divisionsPerBeat

In [ ]:
#EX ULTRAWIDE Q, with 2 Measures match
def ultraWideQ2MeasuresMatchFullTrack(ot, ochroma_, olist, qchroma_, divisionsPerBeat=1):
  dpb = divisionsPerBeat
  scores = list()
  qbeats = qchroma_.shape[0]//divisionsPerBeat
  qmeasures = qbeats // 4
  obeats = ochroma_.shape[0]//divisionsPerBeat
  omeasures = obeats // 4

  for pitch in tqdm([-3, -2, -1, 0, 1, 2, 3]):
    qr = np.roll(qchroma_, pitch, axis=1)
    s = np.matmul(ochroma_, qr.T)
    print(ochroma.shape, qr.shape, s.shape)
    tstart = 0

    gscore = 0
    patterns = list()
    pattern8s = list()
    for len in [8, 4, 2, 1]:
      score = 0
      pattern = list()
      pattern8 = list()
      for qm in range(0,qmeasures, len):
        max         = -1.0
        maxmeasure  = -1
        max8        = -1.0
        maxmeasure8 = -1
        for o in olist:
          for tm in range(o[0],o[1]-len+1, 1):
            x = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+len, dpb), m2i(qm, dpb):m2i(qm+len, dpb)])
            #print(tm, qm, x, m2i(tm, dpb), m2i(qm, dpb))
            if x > max:
              max = x
              maxmeasure = tm
            if tm < 8 and x > max8:
              max8 = x
              maxmeasure8 = tm
        #print(maxbeat, x)
        score += max
        for i in range(len):
          pattern.append(maxmeasure+i)
          pattern8.append(maxmeasure8+i)
      gscore += score
      patterns.append((score, pattern, pattern8))

    if not np.isnan(score):
      scores.append((gscore, primarykey(ot), -pitch, patterns[0][0], patterns[0][1], patterns[0][2], patterns[1][0], patterns[1][1], patterns[1][2], patterns[2][0], patterns[2][1], patterns[2][2], patterns[3][0], patterns[3][1], patterns[3][2]))
    else:
      print('isnan')

  return scores

In [ ]:
#EX ULTRAWIDE Q, with 2 Measures match
def ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochroma_, olist, ofirstm, qchroma_, pitch_, pattern, divisionsPerBeat=1):
  patternind = pattern2ind(pattern)
  dpb = divisionsPerBeat
  scores = list()
  qbeats = qchroma_.shape[0]//divisionsPerBeat
  qmeasures = qbeats // 4
  obeats = ochroma_.shape[0]//divisionsPerBeat
  omeasures = obeats // 4

  print('qbeats=', qbeats, '; qmeasures=', qmeasures, '; pattern=', len(pattern)*4, '; OK=', qmeasures==len(pattern)*4)
  if qmeasures!=len(pattern)*4:
    return

  for pitch in tqdm([pitch_]):
    qr = np.roll(qchroma_, pitch, axis=1)
    s = np.matmul(ochroma_, qr.T)
    print(ochroma.shape, qr.shape, s.shape)

    tstart = 0
    # gscore = 0
    # xpatterns = list()
    # xpattern8s = list()
    length = 4
    score = 0
    # xpattern = list()
    # xpattern8 = list()
    symbolmeasure =  dict()
    symbolmeasureOld =  dict()
    symbolmeasureOldOld =  dict()
    for patternsymbol in patternind.keys():
        patternindices = patternind[patternsymbol]
        maxtmscore = -3.0
        maxtm      = -3
        maxtmscore2 = -2.0
        maxtm2      = -2
        maxtmscore3 = -1.0
        maxtm3      = -1
        for o in olist:
            for tm in range(o[0],o[1]-length+1, 4):
              tmscore = 0
              for pi in patternindices:
                #Pro pattern index = 4 Bars
                qm = pi*4
                x = np.trace(s[tstart+m2i(tm, dpb):tstart+m2i(tm+length, dpb), m2i(qm, dpb):m2i(qm+length, dpb)])
                tmscore += x
                # if qm==4:
                #   print(f'qm={qm}, tm={tm}, score={tmscore}')
              if tmscore > maxtmscore:
                maxtmscore3 = maxtmscore2
                maxtm3 = maxtm2
                maxtmscore2 = maxtmscore
                maxtm2 = maxtm
                maxtmscore = tmscore
                maxtm = tm
                print(patternsymbol, o, tm, '3:', maxtm, maxtm2, maxtm3)
              elif tmscore > maxtmscore2:
                maxtmscore3 = maxtmscore2
                maxtm3 = maxtm2
                maxtmscore2 = tmscore
                maxtm2 = tm
                print(patternsymbol, o, tm, '2:', maxtm, maxtm2, maxtm3)
              elif tmscore > maxtmscore3:
                maxtmscore3 = tmscore
                maxtm3 = tm
                print(patternsymbol, o, tm, '1:', maxtm, maxtm2, maxtm3)
        score += maxtmscore
        symbolmeasure[patternsymbol]=maxtm
        symbolmeasureOld[patternsymbol]=maxtm2
        symbolmeasureOldOld[patternsymbol]=maxtm3
            
    if not np.isnan(score):
      scores.append((score, ot, pitch, ind2pattern(symbolmeasure, pattern, length), symbolmeasure, ind2pattern(symbolmeasureOld, pattern, length), symbolmeasureOld, ind2pattern(symbolmeasureOldOld, pattern, length), symbolmeasureOldOld))
    else:
      print('isnan')

  return scores

In [ ]:
pitch_= ofullsong[2]
ofirstm=1
pattern_=['A', 'A', 'B', 'B', 'C', 'D', 'C', 'D']
oscores = list()
tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][0], olist, ofirstm, qchromas[0][0],pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][1], olist, ofirstm, qchromas[0][0], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][2], olist, ofirstm, qchromas[0][0], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])

tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][0], olist, ofirstm, qchromas[0][1], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][1], olist, ofirstm, qchromas[0][1], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][2], olist, ofirstm, qchromas[0][1], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])

tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][0], olist, ofirstm, qchromas[0][2], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][1], olist, ofirstm, qchromas[0][2], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrackPattern(ot, ochromas[0][2], olist, ofirstm, qchromas[0][2], pitch_, pattern_,  divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])




In [ ]:
oscores = list()
tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][0], olist, qchromas[0][0], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][1], olist, qchromas[0][0], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][2], olist, qchromas[0][0], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])

tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][0], olist,qchromas[0][1], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][1], olist, qchromas[0][1], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][2], olist, qchromas[0][1], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])

tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][0], olist, qchromas[0][2], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][1], olist, qchromas[0][2], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])
tmp = ultraWideQ2MeasuresMatchFullTrack(ot, ochromas[0][2], olist, qchromas[0][2], divisionsPerBeat=1)
tmp.sort(key=lambda s: s[0], reverse=True)
oscores.append(tmp[0])




In [ ]:
oscores.sort(key=lambda s: s[0], reverse=True)

In [ ]:
print(oscores[0])

In [ ]:
loop = loops[0]
for s in oscores:
  if s[2] in [4,6,8] or True:
    print(s[3])
    print(s[5])
    print(s[7])
    outputMix(loop, s, '8', _pattternind=3, loopdb=-12, ovldb=-15)
    # outputMix(loop, s, '8', _pattternind=5, loopdb=-6, ovldb=-15)
    # outputMix(loop, s, '8', _pattternind=7, loopdb=-6, ovldb=-15)
    # outputMix(loop, s, '4', _pattternind=7, loopdb=-15)
    # outputMix(loop, s, '2', _pattternind=10, loopdb=-15)
    # outputMix(loop, s, '1', _pattternind=13, loopdb=-15)


###ExportBeatgridVDJ(ot)

In [ ]:
exportBeatgridVDJ(mbg)

In [ ]:
60/(219-216.31)*4

In [ ]:
exportCueMeasures(ot, [84, 88, 6, 10, 7, 11])

In [ ]:
scoresfm[0]

#MANUAL MODE


---



# Neuer Abschnitt

##pychord

In [ ]:
from pychord import Chord
c = Chord("Ebm")

In [ ]:
c.components()

['F', 'A', 'C', 'D'] f6
['D', 'F', 'A', 'C'] dm7

In [ ]:
Chord('D#m').components()

In [ ]:
Chord('F#6').components()

In [ ]:
Chord('G#m7').components()

In [ ]:
pitches = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B', 'C','Db','D','Eb','E','F','Gb','G','Ab','A','Bb','B'] #, 'cm','c#m','dm','d#m','em','fm','f#m','gm','g#m','a','a#m','bm']

In [ ]:
def createChordPCP(c):
  v = np.zeros(12)
  for note in c.components():
    #print(note, pitches.index(note))
    v[pitches.index(note)%12]=1.
  return normalize(v)

In [ ]:
createChordPCP(Chord('F6'))

In [ ]:
def manualMode(chordseq):
  chords = list()
  for c in chordseq:
    ind = pitches.index(c)
    chords.append(pcps[ind])

  return np.stack(chords)


In [ ]:
F6.components()

In [ ]:
Fis6 = Chord('F#6')
F = Chord('F')
Gism7 = Chord('G#m7')
Dism7 = Chord('D#m7')
Dism = Chord('D#m')
Dm7 = Chord('Dm7')
Dm = Chord('Dm')
Am = Chord('Am')
Am7 = Chord('Am7')
Cissus4 = Chord('C#sus4')
Cissus2 = Chord('C#sus2')
Cis = Chord('C#')
C = Chord('C')
G = Chord('G')
FA = Chord('F/A')
Ebm = Chord('Ebm')
Ebm6 = Chord('Ebm6')
Abm = Chord('Abm')
Db = Chord('Db')
D = Chord('D')
Bb = Chord('Bb')
C7E = Chord('C7/E')

In [ ]:
def mult(c, times):
  return [c for i in range(times)]

In [ ]:
# v1 = [F6, F6, F6, F6, F6, F6, F6, F6, F6, F6, F6, F6, F6, F6, F6, F6]
# v2 = [Gm7, Gm7, Gm7, Gm7, Gm7, Gm7, Gm7, Gm7, F6, F6, F6, F6, F6, F6, F6, F6]
# v3 = [Gm7, Gm7, Gm7, Dm7, Dm7, Dm7, Dm7, Dm7, Gm7, Gm7, Gm7, Dm7, Dm7, Dm7, Dm7, Dm7]
# p1 = [Csus4, Csus4, Csus4, C, Csus2, Csus2, C, C]
# lav = v1+v2+v1+v3+p1+v1+v2

In [ ]:
v1 = mult(Fis6, 16)
v2 = mult(Gism7, 8) + mult(Fis6, 8)
v3 = mult(Gism7, 4) + mult(Dism7, 4) + mult(Gism7, 4) + mult(Dism7, 4)
p1 = [Cissus4, Cissus4, Cissus4, Cis, Cissus2, Cissus2, Cis, Cis]
lav = v1+v2+v1+v3+p1+v1+v2

In [ ]:
v1 = mult(Fis6, 4) + mult(Dism, 4) + mult(Gism7, 4) + mult(Fis6, 4)
v2 = mult(Dism, 12) + mult(Fis6, 4)
v3 = mult(Gism7, 4) + mult(Dism7, 4) + mult(Gism7, 4) + mult(Dism7, 4)
p1 = [Cissus4, Cissus4, Cissus4, Cis, Cissus2, Cissus2, Cis, Cis]
lav = v1+v2+v1+v3+p1+v1+v2

In [ ]:
v0 = mult(Ebm6, 4) + mult(Bb, 4) + mult(Dm7, 4)
v1 = v0 + mult(Am, 4)
v2 = v0 + [C, Dm, C7E, C7E]
c0 = [ F, F, Bb, C, Am7, Am7, Dm, C] + mult(Gm7, 6) + [Am7, Am7]
ngbms = v1 + v2 + c0 + c0

In [ ]:
len(ngbms)

In [ ]:
# v1 = [Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm, Ebm]
# v2 = [Abm, Abm, Abm, Abm, Abm, Abm, Abm, Abm,  Ebm, Ebm, Ebm, Ebm,Ebm, Ebm, Ebm, Ebm]
# v3 = [Abm, Abm, Abm, Abm, Ebm, Ebm, Ebm, Ebm, Abm, Abm, Abm, Abm, Ebm, Ebm, Ebm, Ebm]
# p1 = [Db, Db, Db, Db, Db, Db, Db, Db]
# lav = v1+v2+v1+v3+p1+v1+v2

In [ ]:
#Tarzan Boy intro
i1 = [F, F, F, C, Dm, Dm , Dm , Dm, Bb, Bb, Bb, C, F, F, F, F]
v1 = [Dm, Dm, Bb, Bb, C, C, C, C]
v2 = [Dm, Dm, Bb, Bb, C, C, C7E, C7E]
v3 = [Dm, Dm, C, C, F, F, F, F]
v = v1 + v1 + v2 + v3
print(len(v))
c = [F, F, F, C, Dm7, Dm7, Dm7, Dm7, Bb, Bb, Bb, C, F, F, F, F]
tb = c+c+c+c

In [ ]:
c1 = [C, C, C, C, G, G, G, G, F, F, F, F, FA, FA, FA, FA]
c2 = [C, C, C, C, G, G, G, G, F, F, F, F, D, D, D, D]
hltw = c1 + c2 + c1 + c2

In [ ]:
pcps = list()
for c in hltw:
  #c.transpose(1)
  pcps.append(createChordPCP(c))

In [ ]:
len(pcps)

In [ ]:
qmanual = np.stack(pcps)

In [ ]:
qmanual.shape

In [ ]:
qstring = 'c#m c#m c#m c#m f#m f#m f#m f#m c#m c#m c#m c#m f#m f#m f#m f#m c#m c#m c#m c#m f#m f#m f#m f#m c#m c#m c#m c#m f#m f#m f#m f#m'

In [ ]:
qstring = "Em Em Em Em Am Am F F C C C C G G G G Em Em Em Em Am Am F F C C C C G G G G "

Falco KoMMissar:
v = 'am am G G G G em em  am am G G em em em em '
c = 'D D D D am am am am D D D D  am am am am  D D D D  C C  am am '
qstring = v + v + v + v + c + ' am am G G G G em em'

In [ ]:
#Tarzan Boy
c = ' F F F C dm dm dm dm A# A# A# C F F F F '
v1 = ' dm dm A# A# C C C C '
v2 = ' dm dm A# A# C C A A '
v3 = ' dm dm C C F F F F  '

v = v1 + v1 + v2 + v3


qstring = c + c + v + v + c + c

In [ ]:
len(qstring.split())

In [ ]:
162-34

In [ ]:
all = dur.copy()

In [ ]:
all.extend(moll)

In [ ]:
all

In [ ]:
chordseq = qstring.split()

In [ ]:
chords = list()
for c in chordseq:
  if c in all:
    ind = all.index(c)
    chords.append(freepcps[ind])
    chords.append(freepcps[ind])
    chords.append(freepcps[ind])
    chords.append(freepcps[ind])
  else:
    print("ALARM", c)
  

qmanual = np.stack(chords)

In [ ]:
# trackse = trackf.keys()
# valuese = [trackf[k] for k in trackse]
# trackme = np.concatenate(valuese)

# #scoresfm = ultraWideQ2MeasuresMatch(list(trackse)[0:3], trackme[0:3*96*4*4], qchromas[0][0], divisionsPerBeat=4)
# scoresfm = ultraWideQ2MeasuresPatternMatch(trackse, trackme, qchromas[0][0], pattern=['A', 'A', 'B', 'B', 'C', 'D', 'C', 'D'],  divisionsPerBeat=1)

In [ ]:
qs_[0][1]

In [ ]:
qmanual.shape

In [ ]:
len(qmanual)

In [ ]:
qmanual2 = np.concatenate((qmanual,qmanual), axis=0)

In [ ]:
qmanual2.shape

In [ ]:
trackse = trackm.keys()
#trackse = ['/mnt/c/Users/andre/OneDrive/Music/Library/DB/a-ha - Take on Me (Extended Mix).mp3', '/mnt/c/Users/andre/OneDrive/Music/Library/DB/a-ha - Take on Me (Extended Mix).mp3', '/mnt/c/Users/andre/OneDrive/Music/Library/DB/a-ha - Take on Me (Extended Mix).mp3', '/mnt/c/Users/andre/OneDrive/Music/Library/DB/a-ha - Take on Me (Extended Mix).mp3']
valuese = [trackm[k] for k in trackse]
trackme = np.concatenate(valuese)


_111 = 2
qmanual2 = np.concatenate((qmanual,qmanual), axis=0)
q2 =  np.concatenate((trackm[str(qt)][1*(qs_[0][0]-_111):1*(qs_[0][1]-_111)],trackm[str(qt)][1*(qs_[0][0]-_111):1*(qs_[0][1]-_111)]), axis=0)
q2f =  np.concatenate((trackf[str(qt)][1*(qs_[0][0]-_111):1*(qs_[0][1]-_111)],trackf[str(qt)][1*(qs_[0][0]-_111):1*(qs_[0][1]-_111)]), axis=0)


print('****************************')
print('qm:',len(qmanual), ', q2:',len(q2))

#scoresfm = ultraWideQ2MeasuresMatch2(trackse, trackme,  trackf[str(qt)][1*(qs_[0][0]-_111):1*(qs_[0][1]-_111)], trackm[str(qt)][1*(qs_[0][0]-_111):1*(qs_[0][1]-_111)], qmix=0.6, divisionsPerBeat=1) #qmix=0.63 // trackf[str(qt)][32:5*32]
scoresfm = ultraWideQ2MeasuresMatch2(trackse, trackme,  qmanual2, q2, qmix=0., divisionsPerBeat=1) #qmix=0.63 // trackf[str(qt)][32:5*32]


In [ ]:
# trackse = trackm.keys()
# #trackse = ['/mnt/c/Users/andre/OneDrive/Music/Library/DB/Albert One - Heart On Fire (Special Maxi Mix).mp3', str(qt), '/mnt/c/Users/andre/OneDrive/Music/Library/DB/a-ha - Take on Me (Extended Mix).mp3', '/mnt/c/Users/andre/OneDrive/Music/Library/DB/a-ha - Take on Me (Extended Mix).mp3', '/mnt/c/Users/andre/OneDrive/Music/Library/DB/a-ha - Take on Me (Extended Mix).mp3']
# #trackse = [str(qt), str(qt)]
# valuese = [trackm[k] for k in trackse]
# trackme = np.concatenate(valuese)

# scoresfm = ultraWideQ2MeasuresMatch(trackse, trackme, trackm[str(qt)][32:5*32], divisionsPerBeat=1) #qmix=0.63


In [ ]:
scorefm = score2

In [ ]:
scoresfm.sort(key=lambda s: s[0], reverse=True)

In [ ]:
print(scoresfm[0])
print(scoresfm[1])
print(scoresfm[2])
print(scoresfm[3])
print(scoresfm[4])

In [ ]:
def odd(p):
  for i in range(0, len(p), 4):
    if p[i]==2 or p[i] % 4 != 0:
      print(p, p[i])
      return True
  return False

In [ ]:
qt

In [ ]:
#verbessern: zuest anzeige, dann musik
loop = loops[0]
for s in scoresfm[80:100]:
  if odd(s[4]) or True:
    print(s)
    outputMix(loop, s, '8', _pattternind=4)
    # outputMix(loop, s, '4', _pattternind=7)
    # outputMix(loop, s, '2', _pattternind=10)
    # outputMix(loop, s, '1', _pattternind=13)


In [ ]:
seen = dict()
pos = 0

In [ ]:
loop=(loops[0][0]*2, loops[0][1])
shown = 0


while True:
 


  s = scoresfm[pos]
  pos += 1
  print(s)

  id = s[1]

  if id in seen:
    count = seen[id]
    if count > 0:
      continue
    else:
      count +=1
      seen[id]=count
  else:
    seen[id]=1

  # if s[2] in [4, 6, 8]:
  #   print(lookup[id].attrib['Comments'])  
  #   outputMix(loop, s, scorest[sind])
  #   outputMix(loop, s, scorest[sind], _2nd=True)

    #shown+=1
    # if shown < 100:
    #   continue


  print(getComment(lookup(id)))  
  # if s[1]=='/mnt/c/Users/andre/OneDrive/Music/Library/DB/Alphaville - Sounds Like a Melody (2019 Remaster).mp3':
  #   continue
    
  mix = outputMix(loop, s, 'Manual4', _pattternind=4, showmix=True)
  #mixes.append(mix)
  # outputMix(loop, s, '4', _pattternind=7)
  # outputMix(loop, s, '2<8', _pattternind=11)
  # outputMix(loop, s, '1<8', _pattternind=14)
  shown+=1

  if shown > 20:
    # mixout = AudioSegment.empty()
    # for mix in mixes:
    #   mixout = mixout + mix
    # IPython.display.display(mixout)
    break
    

In [ ]:
score1 = scoresfm[0:1000]
score1 = score1[0:200]


A = set()
A.update([s[1] for s in score1])
len(A)



In [ ]:
score2 = scoresfm[0:1000]
score2 = score2[0:200]

B = set()
B.update([s[1] for s in score2])
len(B)

In [ ]:
len(B.intersection(A))

In [ ]:
628/829

In [ ]:
#p = manualMode(qstring.split())

In [ ]:
def findScore(scores, title):
  for i, s in enumerate(scores):
    if s[1].lower().find(title) > 0:
      print(i, s)

In [ ]:
findScore(scoresfm, 'stride')